<a href="https://colab.research.google.com/github/Billy-Drunkenstein/Jupiter/blob/main/Feature_Validation/test_OLS_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pathlib import Path
import os
from typing import Sequence, Union, Literal
import numpy as np
import pandas as pd
from typing import List, Tuple
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
from __future__ import annotations

In [ ]:
base_dir = "/gpfs/hddfs/shared/tzheng_ryin"
features_dir = "cnfut/cnfut_snap_pool_feather"
targets_dir = "cnfut/cnfut_snap_y_feather"
ref_headers_dir = "cnfut/FeatureHeaders.00000000"
output_dir = "cnfut_meta_matrices"

features_path = os.path.join(base_dir, features_dir)
targets_path = os.path.join(base_dir, targets_dir)
calendar_path = os.path.join(base_dir, "calendar/cn_calendar")
reference_headers_path = os.path.join(base_dir, ref_headers_dir)
output_path = Path(os.path.join(base_dir, output_dir))

In [ ]:
target_cols = ["y60r05"]

# Read

In [ ]:
with open(calendar_path, "r", encoding="utf-8") as f:
    calendar = [int(line.rstrip("\n")) for line in f]

try:
    with open(reference_headers_path, "r", encoding="utf-8") as f:
        ref_headers = [line.rstrip("\n") for line in f]
except FileNotFoundError as e:
    raise FileNotFoundError(
        "Reference Headers Not Found: make sure FeatureHeaders.00000000 exists in cnfut directory"
    ) from e

In [ ]:
def read_meta_matrices(
    output_path: str | Path,
    date: int,
    *,
    ref_headers: Sequence[str],
    target_cols: Sequence[str],
):
    out = Path(output_path)
    p = len(ref_headers)

    headers_path = out / f"FeatureHeaders.{date}"
    xx_path = out / f"XX.{date}.csv"
    corr_path = out / f"FeatureCorr.{date}.csv"

    e_headers = headers_path.exists()
    e_xx = xx_path.exists()
    e_corr = corr_path.exists()
    n_exist = int(e_headers) + int(e_xx) + int(e_corr)

    if n_exist == 0:
        raise FileNotFoundError(
            f"Missing date {date}: none of Headers/XX/Corr exist under {out}"
        )

    if n_exist != 3:
        missing_ids = []
        if not e_headers:
            missing_ids.append("Headers")
        if not e_xx:
            missing_ids.append("XX")
        if not e_corr:
            missing_ids.append("Corr")
        raise FileNotFoundError(f"Broken date {date}: missing {missing_ids}")

    # ---------- Headers ----------
    try:
        with headers_path.open("r", encoding="utf-8") as f:
            headers = [line.strip() for line in f if line.strip()]
    except Exception as e:
        raise IOError(f"Failed reading Headers at {date}") from e

    if headers != list(ref_headers):
        raise ValueError(f"Headers mismatch at {date}")

    # ---------- XX ----------
    try:
        XX_d = pd.read_csv(xx_path, header=None).to_numpy(dtype=np.float32, copy=False)
    except Exception as e:
        raise IOError(f"Failed reading XX at {date}") from e

    exp_shape = (p + 1, p + 1)
    if XX_d.shape != exp_shape:
        raise ValueError(f"XX shape mismatch at {date}: expected {exp_shape}, got {XX_d.shape}")

    if not np.all(XX_d[1:, p] == 0):
        raise ValueError(f"Unexpected nonzero entries in XX last column at {date}")

    # ---------- Corr ----------
    try:
        df = pd.read_csv(corr_path)
    except Exception as e:
        raise IOError(f"Failed reading Corr at {date}") from e

    if "fid" not in df.columns:
        raise ValueError(f"Missing column 'fid' at {date}")

    if df.shape[0] != p:
        raise ValueError(f"Feature row count mismatch at {date}: expected {p}, got {df.shape[0]}")

    fid_ref = [f"F{i}" for i in range(p)]
    if df["fid"].tolist() != fid_ref:
        raise ValueError(f"fid ordering mismatch at {date}")

    prefixes = [f"Y{k}_" for k in range(len(target_cols))]
    for pref in prefixes:
        need = [f"{pref}N", f"{pref}X", f"{pref}XX", f"{pref}Y", f"{pref}YY", f"{pref}XY"]
        miss = [c for c in need if c not in df.columns]
        if miss:
            raise ValueError(f"Missing columns at {date} for {pref}: {miss}")

    return headers, XX_d, df


def assemble_dates(
    calendar: Sequence[int],
    asof: int,
    n: int,
    mode: Literal["in sample", "out of sample"],
) -> List[int]:
    """
    in sample: includes asof date
    out of sample: excludes asof date
    """
    if n < 1:
        raise ValueError("Include at least 1 day")

    if asof not in calendar:
        raise ValueError("Date currently restricted to cn_calendar dates")

    i = calendar.index(asof)

    if mode == "in sample":
        start = i - (n - 1)
        end = i + 1
        if start < 0:
            raise ValueError(f"Insufficient calendar history for asof = {asof} and n = {n}")
        return list(calendar[start:end])

    if mode == "out of sample":
        start = i + 1
        end = i + 1 + n
        if end > len(calendar):
            raise ValueError(f"Insufficient forward calendar for asof = {asof} and n = {n}")
        return list(calendar[start:end])

    raise ValueError(f"Unknown mode: {mode}")


def build_windows(
    output_path: str | Path,
    calendar: Sequence[int],
    asof: int,
    lookback: int,
    lookforward: int,
    *,
    ref_headers: Sequence[str],
    target_cols: Sequence[str],
):
    out = Path(output_path)
    p = len(ref_headers)
    prefixes = [f"Y{k}_" for k in range(len(target_cols))]

    IS_dates = assemble_dates(calendar, asof, lookback, "in sample")
    OOS_dates = assemble_dates(calendar, asof, lookforward, "out of sample")

    IS_xx = np.zeros((p, p), dtype=np.float32)
    IS_stats = {
        "N": {pref: np.int64(0) for pref in prefixes},
        "x_sum": {pref: np.zeros((p,), dtype=np.float32) for pref in prefixes},
        "y_sum": {pref: np.float32(0.0) for pref in prefixes},
        "yy": {pref: np.float32(0.0) for pref in prefixes},
        "xy": {pref: np.zeros((p,), dtype=np.float32) for pref in prefixes},
    }

    for d in IS_dates:
        _, XX_d, df = read_meta_matrices(
            out,
            d,
            ref_headers = ref_headers,
            target_cols = target_cols,
        )

        XTX_d = XX_d[1:, :p]
        if XTX_d.shape != (p, p):
            raise ValueError(f"Bad XTX block at {d}: expected {(p, p)}, got {XTX_d.shape}")

        IS_xx += XTX_d.astype(np.float32, copy = False)

        for pref in prefixes:
            N_d = np.int64(df[f"{pref}N"].iloc[0])
            X_d = df[f"{pref}X"].to_numpy(dtype = np.float32, copy = False)
            Y_d = np.float32(df[f"{pref}Y"].iloc[0])
            YY_d = np.float32(df[f"{pref}YY"].iloc[0])
            XY_d = df[f"{pref}XY"].to_numpy(dtype = np.float32, copy = False)

            if X_d.shape[0] != p or XY_d.shape[0] != p:
                raise ValueError(
                    f"Vector length mismatch at {d} for {pref}: expected {p}, got X = {X_d.shape}, XY = {XY_d.shape}"
                )

            IS_stats["N"][pref] += N_d
            IS_stats["x_sum"][pref] += X_d
            IS_stats["y_sum"][pref] += Y_d
            IS_stats["yy"][pref] += YY_d
            IS_stats["xy"][pref] += XY_d

    OOS_xx = np.zeros((p, p), dtype=np.float32)
    OOS_stats = {
        "N": {pref: np.int64(0) for pref in prefixes},
        "x_sum": {pref: np.zeros((p,), dtype=np.float32) for pref in prefixes},
        "y_sum": {pref: np.float32(0.0) for pref in prefixes},
        "yy": {pref: np.float32(0.0) for pref in prefixes},
        "xy": {pref: np.zeros((p,), dtype=np.float32) for pref in prefixes},
    }

    OOS_daily = {}

    for d in OOS_dates:
        _, XX_d, df = read_meta_matrices(
            out,
            d,
            ref_headers = ref_headers,
            target_cols = target_cols,
        )

        day_xx = np.zeros((p, p), dtype=np.float32)
        day_stats = {
            "N": {pref: np.int64(0) for pref in prefixes},
            "x_sum": {pref: np.zeros((p,), dtype=np.float32) for pref in prefixes},
            "y_sum": {pref: np.float32(0.0) for pref in prefixes},
            "yy": {pref: np.float32(0.0) for pref in prefixes},
            "xy": {pref: np.zeros((p,), dtype=np.float32) for pref in prefixes},
        }

        XTX_d = XX_d[1:, :p]
        if XTX_d.shape != (p, p):
            raise ValueError(f"Bad XTX block at {d}: expected {(p, p)}, got {XTX_d.shape}")

        add_xx = XTX_d.astype(np.float32, copy = False)
        day_xx += add_xx
        OOS_xx += add_xx

        for pref in prefixes:
            N_d = np.int64(df[f"{pref}N"].iloc[0])
            X_d = df[f"{pref}X"].to_numpy(dtype = np.float32, copy = False)
            Y_d = np.float32(df[f"{pref}Y"].iloc[0])
            YY_d = np.float32(df[f"{pref}YY"].iloc[0])
            XY_d = df[f"{pref}XY"].to_numpy(dtype = np.float32, copy = False)

            if X_d.shape[0] != p or XY_d.shape[0] != p:
                raise ValueError(
                    f"Vector length mismatch at {d} for {pref}: expected {p}, got X = {X_d.shape}, XY = {XY_d.shape}"
                )

            day_stats["N"][pref] += N_d
            day_stats["x_sum"][pref] += X_d
            day_stats["y_sum"][pref] += Y_d
            day_stats["yy"][pref] += YY_d
            day_stats["xy"][pref] += XY_d

            OOS_stats["N"][pref] += N_d
            OOS_stats["x_sum"][pref] += X_d
            OOS_stats["y_sum"][pref] += Y_d
            OOS_stats["yy"][pref] += YY_d
            OOS_stats["xy"][pref] += XY_d

        OOS_daily[d] = (day_xx, day_stats)

    return IS_xx, IS_stats, OOS_xx, OOS_stats, OOS_daily


def fit_from_meta(
    IS_xx: np.ndarray,
    IS_stats: dict,
    *,
    target_cols: list[str],
):
    prefixes = [f"Y{k}_" for k in range(len(target_cols))]

    p = IS_xx.shape[0]
    if IS_xx.shape != (p, p):
        raise ValueError(f"IS_xx must be square, got {IS_xx.shape}")

    betas = {}
    IS_metrics = {}

    xx64 = IS_xx.astype(np.float64, copy = False)

    for pref in prefixes:
        n = int(IS_stats["N"][pref])
        if n <= 0:
            raise ValueError(f"{pref}N must be > 0, got {n}")

        xs = IS_stats["x_sum"][pref].astype(np.float64, copy = False)
        ys = float(IS_stats["y_sum"][pref])
        yty = float(IS_stats["yy"][pref])
        xty = IS_stats["xy"][pref].astype(np.float64, copy = False)

        if xs.shape != (p, ):
            raise ValueError(f"{pref}x_sum shape mismatch: expected {(p, )}, got {xs.shape}")
        if xty.shape != (p, ):
            raise ValueError(f"{pref}xy shape mismatch: expected {(p, )}, got {xty.shape}")

        G = np.empty((p + 1, p + 1), dtype = np.float64)
        G[0, 0] = n
        G[0, 1:] = xs
        G[1:, 0] = xs
        G[1:, 1:] = xx64

        g = np.empty((p + 1, ), dtype = np.float64)
        g[0] = ys
        g[1:] = xty

        beta = np.linalg.solve(G, g)

        betaTg = float(beta @ g)
        sse = yty - betaTg
        if sse < 0 and sse > -1e-6 * max(1.0, yty):
            sse = 0.0

        mse = sse / n
        rmse = float(np.sqrt(mse))

        ybar = ys / n
        tss = yty - n * (ybar ** 2)
        if tss <= 0:
            r2 = np.nan
        else:
            r2 = 1.0 - (sse / tss)

        betas[pref] = beta
        IS_metrics[pref] = {
            "N": n,
            "SSE": float(sse),
            "MSE": float(mse),
            "RMSE": float(rmse),
            "R2": float(r2) if np.isfinite(r2) else r2,
        }

    return betas, IS_metrics


def score_one_window(
    beta: np.ndarray,
    xx: np.ndarray,
    stats: dict,
    *,
    pref: str,
):
    xx = np.asarray(xx)
    p = xx.shape[0]
    if xx.shape != (p, p):
        raise ValueError(f"xx must be square, got {xx.shape}")

    beta = np.array(beta, dtype = np.float64)
    if beta.shape != (p + 1, ):
        raise ValueError(f"beta shape mismatch: expected {(p + 1, )}, got {beta.shape}")

    n = int(stats["N"][pref])
    if n <= 0:
        raise ValueError("Must have at least some samples")

    xs = stats["x_sum"][pref].astype(np.float64, copy = False)
    ys = float(stats["y_sum"][pref])
    yty = float(stats["yy"][pref])
    xty = stats["xy"][pref].astype(np.float64, copy = False)

    if xs.shape != (p, ):
        raise ValueError(f"{pref}x_sum shape mismatch: expected {(p, )}, got {xs.shape}")
    if xty.shape != (p, ):
        raise ValueError(f"{pref}xy shape mismatch: expected {(p, )}, got {xty.shape}")

    b0 = float(beta[0])
    b = beta[1:]

    xx64 = xx.astype(np.float64, copy = False)

    betaT_Xty = b0 * ys + float(b @ xty)
    betaT_XtX_beta = (b0 * b0) * n + 2.0 * b0 * float(b @ xs) + float(b @ (xx64 @ b))

    sse = yty - 2.0 * betaT_Xty + betaT_XtX_beta
    if sse < 0 and sse > -1e-6 * max(1.0, yty):
        sse = 0.0

    mse = sse / n
    rmse = float(np.sqrt(mse))

    ybar = ys / n
    tss = yty - n * (ybar ** 2)
    if tss <= 0:
        r2 = np.nan
    else:
        r2 = 1.0 - (sse / tss)

    return {
        "N": n,
        "SSE": float(sse),
        "MSE": float(mse),
        "RMSE": float(rmse),
        "R2": float(r2) if np.isfinite(r2) else r2,
    }


def test_from_meta(
    betas: dict,
    OOS_xx: np.ndarray,
    OOS_stats: dict,
    OOS_daily: dict,
    *,
    target_cols: list[str],
):
    prefixes = [f"Y{k}_" for k in range(len(target_cols))]
    out = {"agg": {}}

    for pref in prefixes:
        if pref not in betas:
            raise KeyError(f"Missing beta for {pref}")

        out["agg"][pref] = score_one_window(
            beta = betas[pref],
            xx = OOS_xx,
            stats = OOS_stats,
            pref = pref,
        )

    if OOS_daily is not None:
        out["daily"] = {pref: {} for pref in prefixes}
        OOS_dates = sorted(OOS_daily.keys())

        for j, d in enumerate(OOS_dates, start = 1):
            day_xx, day_stats = OOS_daily[d]
            key = f"t+{j}"

            for pref in prefixes:
                out["daily"][pref][key] = score_one_window(
                    beta = betas[pref],
                    xx = day_xx,
                    stats = day_stats,
                    pref = pref,
                )

    return out

In [ ]:
# asof = 20251020
# lookback = 30
# lookforward = 5
# target_cols = ["y60r05"]

# IS_xx, IS_stats, OOS_xx, OOS_stats, OOS_daily = build_windows(
#     output_path = output_path,
#     calendar = calendar,
#     asof = asof,
#     lookback = lookback,
#     lookforward = lookforward,
#     ref_headers = ref_headers,
#     target_cols = target_cols,
# )

# p = len(ref_headers)
# prefixes = [f"Y{k}_" for k in range(len(target_cols))]

# assert IS_xx.shape == (p, p)
# assert IS_xx.dtype == np.float32
# assert OOS_xx.shape == (p, p)
# assert OOS_xx.dtype == np.float32

# assert isinstance(OOS_daily, dict)
# assert len(OOS_daily) == lookforward

# OOS_dates = assemble_dates(calendar, asof, lookforward, "out of sample")
# assert sorted(OOS_daily.keys()) == OOS_dates

# for pref in prefixes:
#     assert pref in IS_stats["N"]
#     assert pref in IS_stats["x_sum"]
#     assert pref in IS_stats["y_sum"]
#     assert pref in IS_stats["yy"]
#     assert pref in IS_stats["xy"]

#     assert pref in OOS_stats["N"]
#     assert pref in OOS_stats["x_sum"]
#     assert pref in OOS_stats["y_sum"]
#     assert pref in OOS_stats["yy"]
#     assert pref in OOS_stats["xy"]

#     assert isinstance(IS_stats["N"][pref], (np.integer, int))
#     assert isinstance(OOS_stats["N"][pref], (np.integer, int))
#     assert int(IS_stats["N"][pref]) > 0
#     assert int(OOS_stats["N"][pref]) > 0

#     assert IS_stats["x_sum"][pref].shape == (p, )
#     assert IS_stats["x_sum"][pref].dtype == np.float32
#     assert IS_stats["xy"][pref].shape == (p, )
#     assert IS_stats["xy"][pref].dtype == np.float32

#     assert OOS_stats["x_sum"][pref].shape == (p, )
#     assert OOS_stats["x_sum"][pref].dtype == np.float32
#     assert OOS_stats["xy"][pref].shape == (p, )
#     assert OOS_stats["xy"][pref].dtype == np.float32

# for d in OOS_dates:
#     day_xx, day_stats = OOS_daily[d]
#     assert day_xx.shape == (p, p)
#     assert day_xx.dtype == np.float32
#     for pref in prefixes:
#         assert int(day_stats["N"][pref]) > 0
#         assert day_stats["x_sum"][pref].shape == (p, )
#         assert day_stats["x_sum"][pref].dtype == np.float32
#         assert day_stats["xy"][pref].shape == (p, )
#         assert day_stats["xy"][pref].dtype == np.float32

# for pref in prefixes:
#     sum_daily_N = sum(int(OOS_daily[d][1]["N"][pref]) for d in OOS_dates)
#     assert int(OOS_stats["N"][pref]) == sum_daily_N

#     sum_daily_x_sum = np.zeros((p, ), dtype = np.float32)
#     sum_daily_xy = np.zeros((p, ), dtype = np.float32)
#     for d in OOS_dates:
#         sum_daily_x_sum += OOS_daily[d][1]["x_sum"][pref]
#         sum_daily_xy += OOS_daily[d][1]["xy"][pref]

#     assert np.allclose(OOS_stats["x_sum"][pref], sum_daily_x_sum, rtol = 0, atol = 0)
#     assert np.allclose(OOS_stats["xy"][pref], sum_daily_xy, rtol = 0, atol = 0)

# sum_daily_xx = np.zeros((p, p), dtype = np.float32)
# for d in OOS_dates:
#     sum_daily_xx += OOS_daily[d][0]

# assert np.allclose(OOS_xx, sum_daily_xx, rtol = 0, atol = 0)


In [ ]:
asof = 20251020
lookback = 30
lookforward = 5

IS_xx, IS_stats, OOS_xx, OOS_stats, OOS_daily = build_windows(
    output_path = output_path,
    calendar = calendar,
    asof = asof,
    lookback = lookback,
    lookforward = lookforward,
    ref_headers = ref_headers,
    target_cols = target_cols,
)

betas, IS_metrics = fit_from_meta(
    IS_xx,
    IS_stats,
    target_cols = target_cols,
)

OOS_result = test_from_meta(
    betas = betas,
    OOS_xx = OOS_xx,
    OOS_stats = OOS_stats,
    OOS_daily = OOS_daily,
    target_cols = target_cols,
)